# Prep script

In [30]:
import time
import pylast
import hashlib
from datetime import date
from datetime import datetime
import requests
import pandas as pd, requests, time, json, os.path
import logging as lg, datetime as dt
import pandas as pd, numpy as np, matplotlib.pyplot as plt, logging as lg
import time, requests, re, json, os.path, datetime as dt
from geopy.distance import great_circle
import mpl_toolkits
import sys
import os
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials

In [33]:
%run -i 'functions/all_recently_played_music.py'
%run -i 'functions/get_locations.py'
%run -i 'functions/geocode_locations.py'

<br>

# Set-up API call information

### Last.FM

In [4]:
# Enter your user details
user_name = 'xxx'
user_password = 'xxx'
api_key = 'xxx'
api_secret = 'xx'

# set-up hash / API call
password_hash = pylast.md5(user_password)
network = pylast.LastFMNetwork(api_key = api_key, api_secret =  api_secret, username = user_name, password_hash = password_hash)
user = pylast.User(user_name, network)

### Spotify

In [5]:
cid ="xxx" 
secret = "xxx"
sp = spotipy #spotipy.Spotify()

<br>

# Get all music played

In [12]:
# just gets latest 5 as a check
recent_tracks = user.get_recent_tracks(limit = 5, cacheable=True)

In [13]:
recent_tracks


[PlayedTrack(track=pylast.Track('Kara Marni', 'Trippin', pylast.LastFMNetwork('040075c038c9f0e9200581b60723f860', '994d7d88b34019cb9c337fe179ac0f67', 'ofdzyuBopclyl7-KfG3ss4V2AP0-huLO', 'pollystewart', 'cc71bfda7b314104eb54e2c8fb80b79e')), album='Trippin', playback_date='25 Mar 2021, 17:52', timestamp='1616694758'),
 PlayedTrack(track=pylast.Track('Lauv', 'I Like Me Better', pylast.LastFMNetwork('040075c038c9f0e9200581b60723f860', '994d7d88b34019cb9c337fe179ac0f67', 'ofdzyuBopclyl7-KfG3ss4V2AP0-huLO', 'pollystewart', 'cc71bfda7b314104eb54e2c8fb80b79e')), album='I met you when I was 18. (the playlist)', playback_date='25 Mar 2021, 17:46', timestamp='1616694419'),
 PlayedTrack(track=pylast.Track('Bebe Rexha', "Baby, I'm Jealous (feat. Doja Cat)", pylast.LastFMNetwork('040075c038c9f0e9200581b60723f860', '994d7d88b34019cb9c337fe179ac0f67', 'ofdzyuBopclyl7-KfG3ss4V2AP0-huLO', 'pollystewart', 'cc71bfda7b314104eb54e2c8fb80b79e')), album="Baby, I'm Jealous (feat. Doja Cat)", playback_date='25 

In [16]:
# gets max 999 tracks at a time
tracks = get_all_recent_tracks(user = user, network = network, limit_value = 100)
tracks.head()

,artist,track,album,timestamp,track_duration,listener_count,playcount
0,Kara Marni,Trippin,Trippin,2021-03-25 18:52:38,172000,4008,20068
1,Lauv,I Like Me Better,I met you when I was 18. (the playlist),2021-03-25 18:46:59,197000,309421,2623118
2,Bebe Rexha,"Baby, I'm Jealous (feat. Doja Cat)","Baby, I'm Jealous (feat. Doja Cat)",2021-03-25 18:43:54,175000,119676,1086894
3,Lauv,I Like Me Better,I met you when I was 18. (the playlist),2021-03-25 17:16:04,197000,309421,2623118
4,Bebe Rexha,"Baby, I'm Jealous (feat. Doja Cat)","Baby, I'm Jealous (feat. Doja Cat)",2021-03-25 17:13:09,175000,119676,1086894


In [18]:
tracks_w_tags = get_all_tags(tracks, network)

In [19]:
tracks_w_tags

,artist,track,album,timestamp,track_duration,listener_count,playcount,all_artist_tags,top_3_artist_tags,all_track_tags,top_3_track_tags
0,Kara Marni,Trippin,Trippin,2021-03-25 18:52:38,172000,4008,20068,"[Rnb, Pop, Soul, Acoustic, Female, British, Si...","[Rnb, Pop, Soul]","[Great Quality Stuff, 2021 Single]","[Great Quality Stuff, 2021 Single]"
1,Lauv,I Like Me Better,I met you when I was 18. (the playlist),2021-03-25 18:46:59,197000,309421,2623118,"[Indie Pop, Pop, Indie, Jazz, American, Male V...","[Indie Pop, Pop, Indie]","[Indie, Linedance 2017, To All The Boys Ive Lo...","[Indie, Linedance 2017, To All The Boys Ive Lo..."
2,Bebe Rexha,"Baby, I'm Jealous (feat. Doja Cat)","Baby, I'm Jealous (feat. Doja Cat)",2021-03-25 18:43:54,175000,119676,1086894,"[Pop, Female Vocalists, Electronic, Singer-Son...","[Pop, Female Vocalists, Electronic]","[Pop, 2020, Electronic, Urban, Rnb, Hit, Infat...","[Pop, 2020, Electronic]"
3,Lauv,I Like Me Better,I met you when I was 18. (the playlist),2021-03-25 17:16:04,197000,309421,2623118,"[Indie Pop, Pop, Indie, Jazz, American, Male V...","[Indie Pop, Pop, Indie]","[Indie, Linedance 2017, To All The Boys Ive Lo...","[Indie, Linedance 2017, To All The Boys Ive Lo..."
4,Bebe Rexha,"Baby, I'm Jealous (feat. Doja Cat)","Baby, I'm Jealous (feat. Doja Cat)",2021-03-25 17:13:09,175000,119676,1086894,"[Pop, Female Vocalists, Electronic, Singer-Son...","[Pop, Female Vocalists, Electronic]","[Pop, 2020, Electronic, Urban, Rnb, Hit, Infat...","[Pop, 2020, Electronic]"
...,...,...,...,...,...,...,...,...,...,...,...
95,Vitamin String Quartet,"thank u, next",Bridgerton (Covers from the Netflix Original S...,2021-03-24 12:50:54,209000,13301,43378,"[Instrumental, Classical, Covers, Strings, Tri...","[Instrumental, Classical, Covers]",[],[]
96,CeeLo Green,I Want You (Hold On To Love),The Lady Killer (The Platinum Edition),2021-03-24 12:47:23,0,5853,31844,"[Soul, Funk, Hip-Hop, Seen Live, Rnb, Hip Hop,...","[Soul, Funk, Hip-Hop]",[Strawberry Lemonade],[Strawberry Lemonade]
97,Doja Cat,Like That (feat. Gucci Mane),Hot Pink,2021-03-24 12:44:40,163000,241466,2726721,"[Pop, Rnb, Rap, Hip-Hop, Female Vocalists, Hip...","[Pop, Rnb, Rap]","[Rnb, Hip Hop, Hip-Hop, Pop, Urban, American, ...","[Rnb, Hip Hop, Hip-Hop]"
98,Camila Cabello,She Loves Control,Camila,2021-03-24 12:39:39,177000,138955,1557584,"[Pop, Female Vocalists, Cuba, Cuban, Dance, Se...","[Pop, Female Vocalists, Cuba]","[Pop, Dancehall, Dance, Rnb, Catchy, Bop, Lati...","[Pop, Dancehall, Dance]"


In [20]:
tracks_w_tags.to_csv('data/raw_data/music_with_tags_20210807.csv', index = False)

<br>

# Get MBID of all artists/tracks/albums

In [21]:
df_mbid = get_mbid_of_all_rows(tracks_w_tags, network)

In [22]:
df_mbid.to_csv('data/raw_data/music_with_mbid_20210807.csv', index = False)

<br>

# Get Locations & MusicBrainz Info

In [23]:
all_data = find_all_locations(df_mbid)

get_artist_by_id error: {'gender': 'Female', 'sort-name': 'Rexha, Bebe', 'area': {'type-id': None, 'sort-name': 'United States', 'iso-3166-1-codes': ['US'], 'name': 'United States', 'disambiguation': '', 'id': '489ce91b-6658-3307-9877-795b68554c98', 'type': None}, 'country': 'US', 'id': 'db8fad3a-e131-47a1-8782-c2ee93708cdd', 'end_area': None, 'type': 'Person', 'isnis': ['0000000434296460'], 'ipis': ['00550052885', '00618131569'], 'name': 'Bebe Rexha', 'disambiguation': '', 'life-span': {'end': None, 'ended': False, 'begin': '1989-08-30'}, 'end-area': None, 'gender-id': '93452b5a-a947-30c8-934f-6a4056b151c2', 'begin_area': {'name': 'New York', 'sort-name': 'New York', 'type-id': None, 'type': None, 'id': '74e50e58-5deb-4b99-93a2-decbb365c07f', 'disambiguation': ''}, 'begin-area': {'name': 'New York', 'sort-name': 'New York', 'type-id': None, 'type': None, 'id': '74e50e58-5deb-4b99-93a2-decbb365c07f', 'disambiguation': ''}, 'type-id': 'b6e035f4-3ce9-331c-97df-83397230b0df'}


there are 66 unique artists to get details for


row #0 failed: 'NoneType' object is not subscriptable
get_artist_by_id error: {'gender-id': '93452b5a-a947-30c8-934f-6a4056b151c2', 'name': 'Doja Cat', 'end-area': None, 'ipis': ['00361728553'], 'begin_area': {'name': 'Los Angeles', 'id': '1f40c6e1-47ba-4e35-996f-fe6ee5840e62', 'type-id': None, 'disambiguation': '', 'type': None, 'sort-name': 'Los Angeles'}, 'isnis': ['0000000465726144'], 'country': 'US', 'disambiguation': '', 'gender': 'Female', 'life-span': {'begin': '1995-10-21', 'ended': False, 'end': None}, 'id': '5df62a88-cac9-490a-b62c-c7c88f4020f4', 'area': {'iso-3166-1-codes': ['US'], 'disambiguation': '', 'type-id': None, 'type': None, 'sort-name': 'United States', 'name': 'United States', 'id': '489ce91b-6658-3307-9877-795b68554c98'}, 'begin-area': {'name': 'Los Angeles', 'id': '1f40c6e1-47ba-4e35-996f-fe6ee5840e62', 'type-id': None, 'disambiguation': '', 'type': None, 'sort-name': 'Los Angeles'}, 'end_area': None, 'type': 'Person', 'type-id': 'b6e035f4-3ce9-331c-97df-833972

row #8 failed: 'NoneType' object is not subscriptable
get_artist_by_id error: {'name': '5 Seconds of Summer', 'ipis': [], 'gender-id': None, 'life-span': {'ended': False, 'begin': '2011', 'end': None}, 'end-area': None, 'disambiguation': '', 'area': {'type': None, 'id': '106e0bec-b638-3b37-b731-f53d507dc00e', 'disambiguation': '', 'sort-name': 'Australia', 'iso-3166-1-codes': ['AU'], 'name': 'Australia', 'type-id': None}, 'sort-name': '5 Seconds of Summer', 'gender': None, 'isnis': [], 'type': 'Group', 'end_area': None, 'id': '830e5c4e-6b7d-431d-86ab-00c751281dc5', 'country': 'AU', 'type-id': 'e431f5f6-b5d2-343d-8b36-72607fffb74b', 'begin-area': {'sort-name': 'Sydney', 'name': 'Sydney', 'type-id': None, 'type': None, 'disambiguation': '', 'id': '3f179da4-83c6-4a28-a627-e46b4a8ff1ed'}, 'begin_area': {'sort-name': 'Sydney', 'name': 'Sydney', 'type-id': None, 'type': None, 'disambiguation': '', 'id': '3f179da4-83c6-4a28-a627-e46b4a8ff1ed'}}
row #9 failed: [Errno 2] No such file or directo

row #29 failed: [Errno 2] No such file or directory: 'place_data/artist_cache.js'
get_artist_by_id error: {'error': 'Your requests are exceeding the allowable rate limit. Please see http://wiki.musicbrainz.org/XMLWebService for more information.'}
row #30 failed: 'NoneType' object is not subscriptable
get_artist_by_id error: {'error': 'Your requests are exceeding the allowable rate limit. Please see http://wiki.musicbrainz.org/XMLWebService for more information.'}
row #31 failed: 'NoneType' object is not subscriptable
get_artist_by_id error: {'error': 'Your requests are exceeding the allowable rate limit. Please see http://wiki.musicbrainz.org/XMLWebService for more information.'}
row #32 failed: 'NoneType' object is not subscriptable
get_artist_by_id error: {'error': 'Your requests are exceeding the allowable rate limit. Please see http://wiki.musicbrainz.org/XMLWebService for more information.'}
row #33 failed: 'NoneType' object is not subscriptable
get_artist_by_id error: {'error': 

row #64 failed: 'NoneType' object is not subscriptable
get_artist_by_id error: {'type-id': 'e431f5f6-b5d2-343d-8b36-72607fffb74b', 'begin_area': None, 'begin-area': None, 'disambiguation': '', 'life-span': {'begin': '1999', 'ended': False, 'end': None}, 'end-area': None, 'gender-id': None, 'ipis': [], 'name': 'Vitamin String Quartet', 'country': 'US', 'end_area': None, 'id': '56cfe6aa-b464-43d2-bec7-8ec52f313e7d', 'isnis': [], 'type': 'Group', 'gender': None, 'sort-name': 'Vitamin String Quartet', 'area': {'name': 'United States', 'iso-3166-1-codes': ['US'], 'sort-name': 'United States', 'type-id': None, 'type': None, 'disambiguation': '', 'id': '489ce91b-6658-3307-9877-795b68554c98'}}
row #65 failed: 'NoneType' object is not subscriptable
row #0 failed: 'NoneType' object is not subscriptable
row #1 failed: 'NoneType' object is not subscriptable
row #2 failed: 'NoneType' object is not subscriptable
row #3 failed: 'NoneType' object is not subscriptable
row #4 failed: 'NoneType' object i

processed 66 artists in 10.43 seconds and saved csv
66 artists to retry
processed 66 artists in 0.05 seconds and saved csv
we will attempt to get place full names for 0 place IDs
finished getting place full names from place IDs in 0.00 seconds


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [24]:
all_data.to_csv("data/raw_data/music_with_places_20210807.csv", index = False)

<br>

# Get lat/lon coordinates

In [25]:
music_with_lat_lon = get_all_latlons_for_music(all_data, ignore_country_if_more_detail = False)

100 total artists
0 unique places
geocoded 0 addresses in 0.00 seconds
received 0 non-null lat-longs
0 addresses still lack lat-long
geocoded 0 addresses in 0.00 seconds
received 0 non-null lat-longs


In [26]:
music_with_lat_lon.to_csv("data/raw_data/music_with_places_latlon_20210807.csv", index = False)

<br>

# Get info from Spotify

In [36]:
spotify_data = get_spotipy_id(music_with_lat_lon, cid = cid, secret = secret)
spotify_data = spotify_data.drop_duplicates(["artist", "track"])

In [37]:
spotify_data_info = get_spotify_data(spotify_data, batchsize_value = 100,  cid = cid, secret = secret)

In [38]:
spotify_info = pd.merge(spotify_data, spotify_data_info, on = "track_id", how = "left", validate = "one_to_one" )

In [39]:
spotify_info = spotify_info[['artist', 'track', 'popularity', 'acousticness', 'danceability', 'duration_ms', 
                             'energy','instrumentalness', 'liveness','loudness','mode','speechiness','tempo',
                             'time_signature', 'valence']]

In [40]:
music_with_all_features = pd.merge(music_with_lat_lon, spotify_info, on = ["artist", "track"], how = "left", validate = "many_to_one" )


In [41]:
music_with_all_features.to_csv("data/raw_data/music_with_all_features_20210807.csv", index = False)